## 1. Load voxel model and preview

In [ ]:
from setup.voxel_setup import setup_voxel_scene, NUM_XYZ
from common.plot import Plotter
from simulation.simulator import get_irrad_loc_dir, compute_ior_gradient
from simulation.simulate_utils import remove_under_floor

import taichi as ti
from scipy import ndimage

# debug=True to check boundary access
ti.init(arch=ti.gpu)
scene, floor_height = setup_voxel_scene()

sampler_multiplier = 7
to_load_save = True
plotter = Plotter(sampler_multiplier, floor_height)

scene.ior = ndimage.gaussian_filter(scene.ior, sigma=3.0, radius=1)
scene.ior = remove_under_floor(scene.ior, floor_height)
scene.gradient = compute_ior_gradient(scene.ior)
scene.irradiance, scene.local_diretion = get_irrad_loc_dir(scene, sampler_multiplier, to_load_save=to_load_save, plotter=plotter)

scene.display()

## 3. Perform light simulation

In [ ]:
from common.plot import Plotter
from data.octree import Octree
from data.mlp import MLP
from data.siren import SirenFitter, siren_post_process
from simulation.simulator import normalize_by_max, get_irrad_loc_dir, compute_ior_gradient
from scipy import ndimage


%load_ext autoreload
%autoreload 2
# May comment it because the compatibility of this extension is not good
%matplotlib widget 

In [ ]:
sampler_multiplier = 7
to_load_save = True

plotter = Plotter(sampler_multiplier, floor_height)

scene.ior = ndimage.gaussian_filter(scene.ior, sigma=3.0, radius=1)
scene.gradient = compute_ior_gradient(scene.ior)
scene.irradiance, scene.local_diretion = get_irrad_loc_dir(scene, sampler_multiplier, to_load_save=to_load_save, plotter=plotter)

plotter.plot_irradiance_slices(scene.irradiance[:, floor_height:, :], threshold=3, num_slices=4, z_start=30, z_end=100)

In [ ]:
# plotter.plot_wavefront(scene.ior, None, None)
# plotter.plot_gradient(scene.gradient, threshold=0.2)
# plotter.plot_irradiance_grid(scene.irradiance)
# plotter.plot_local_direction_grid_slices(scene.local_diretion[:, floor_height:, :], num_slices=4, z_start=30, z_end=100)

## 4. Fit irradiance using SIREN 

In [ ]:
siren_fitter= SirenFitter(scene.irradiance, floor_height, sampler_multiplier,
                     hidden_features=256, hidden_layers=3, omega=24)
siren_fitter.fit(total_epochs=24, batch_size=20000, lr=5e-4)

In [ ]:
siren_res = siren_fitter.infer()
plotter.plot_irradiance_slices(siren_res, threshold=3, num_slices=4, z_start=30, z_end=100)
siren_res.shape

In [ ]:
corrected_siren_res = siren_post_process(siren_res, gamma=0.6)
plotter.plot_irradiance_slices(corrected_siren_res, threshold=3, num_slices=4, z_start=30, z_end=100)

## 5. Fit irradiance using MLP

In [ ]:
mlp = MLP(scene.irradiance, floor_height, NUM_XYZ, sampler_multiplier, num_epoches=80)

In [ ]:
# Visualize the predicted irradiance field
predicted_irradiance = mlp.predict(pad=True)
plotter.plot_irradiance_slices(predicted_irradiance, threshold=3, 
                               num_slices=4, z_start=30, z_end=100)

## 6. Store irradiance in octree

In [ ]:
octree = Octree(threshold=4)
octree.construct(scene.irradiance)
print(f"Number of nodes: {len(octree)}")
print(f"Octree Memory usage: {octree.__sizeof__()} bytes")
print(f"In comparison, NumPy Storage Usage: {scene.irradiance.nbytes} bytes")
octree.visualize(plotter, num_slices=4, z_start=30, z_end=100)

In [ ]:
testor = octree.init_empty_grid()
octree.fill_grid(octree.root, testor, 0, 0, 0, octree.grid_size)
plotter.plot_irradiance_slices(testor, threshold=3, num_slices=4, z_start=30, z_end=100)

In [ ]:
x, y, z = 60, 60, 60
value = octree.query(x, y, z)
value